<a href="https://colab.research.google.com/github/ppppxxzz/DC_BOT_colab/blob/main/LawBot02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 掛載 Google Drive 到指定目錄
from google.colab import drive
import os
def mount_google_drive():
    """掛載 Google Drive 以存取其檔案。"""
    print("正在掛載 Google Drive...")
    drive.mount('/content/drive/MyDrive/Colab Notebooks/DC_BOT')
    print("Google Drive 掛載成功！檔案位於 '/content/drive/My Drive'。")

# 設定根目錄（您所有檔案均放在此目錄下）
BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/DC_BOT'
os.chdir(BASE_PATH)
current_directory = os.getcwd()
print("當前工作目錄:", current_directory)

# 安裝必要套件（如果尚未安裝的話）
!pip install --upgrade openai==0.28.0 faiss-cpu discord.py

# 匯入所需模組
import json
import openai
import numpy as np
import faiss

# 定義讀取 JSON 檔案的共用函數
def load_json(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"錯誤：找不到 {file_path}，請確認檔案存在。")
        exit(1)
    except json.JSONDecodeError:
        print(f"錯誤：無法解析 {file_path} 的 JSON，請檢查格式。")
        exit(1)

# 載入設定檔（此處的 config.json 必須放在 BASE_PATH 下）
CONFIG_FILE = os.path.join(BASE_PATH, 'config.json')
config = load_json(CONFIG_FILE)
OPENAI_API_KEY = config.get('openai_api_key')
if not OPENAI_API_KEY:
    print("錯誤：config.json 中缺少 openai_api_key 的設定。")
    exit(1)

# 設定 OpenAI API 金鑰
openai.api_key = OPENAI_API_KEY

print("環境設置完成！")


當前工作目錄: /content/drive/MyDrive/Colab Notebooks/DC_BOT
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 886.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1
環境設置完成！


In [6]:
# 設定知識庫資料夾路徑（放在 BASE_PATH 下）
KNOWLEDGE_BASE_FOLDER = os.path.join(BASE_PATH, 'knowledge_base')

# 定義取得嵌入向量的函數
def get_embedding(text):
    try:
        response = openai.Embedding.create(
            input=[text],
            model="text-embedding-ada-002"
        )
        return response['data'][0]['embedding']
    except openai.OpenAIError as e:
        print(f"OpenAI API 錯誤：{e}")
        return None

# 從知識庫資料夾中讀取所有 JSON 檔案，並整理成文件列表
def load_knowledge_base(folder_path):
    knowledge_base = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):  # 只處理 JSON 檔
            file_path = os.path.join(folder_path, file_name)
            try:
                data = load_json(file_path)
                for item in data:
                    # 假設每個條目包含 act、article 與 content 三個欄位
                    knowledge_base.append(f"{item['act']} {item['article']}: {item['content']}")
            except KeyError as e:
                print(f"{file_name} 缺少必要的鍵：{e}")
            except Exception as e:
                print(f"處理 {file_name} 時發生錯誤：{e}")
    return knowledge_base

# 載入知識庫
knowledge_base = load_knowledge_base(KNOWLEDGE_BASE_FOLDER)
if not knowledge_base:
    print("錯誤：知識庫為空，請檢查資料夾內容。")
    exit(1)

# 生成每個文件的嵌入向量
embeddings = []
total_docs = len(knowledge_base)
for idx, doc in enumerate(knowledge_base):
    embedding = get_embedding(doc)
    if embedding is not None:
        embeddings.append(embedding)
    else:
        # 若嵌入失敗，補上 1536 維零向量（text-embedding-ada-002 模型維度為 1536）
        embeddings.append([0.0] * 1536)
        print(f"[FAILURE] ({idx + 1}/{total_docs}) 嵌入失敗：{doc}")

# 轉換為 numpy 陣列
embeddings = np.array(embeddings).astype('float32')

# 儲存嵌入向量與知識庫內容到 BASE_PATH 下
np.save(os.path.join(BASE_PATH, "embeddings.npy"), embeddings)
with open(os.path.join(BASE_PATH, "knowledge_base.json"), "w", encoding="utf-8") as f:
    json.dump(knowledge_base, f, ensure_ascii=False, indent=4)

print("所有嵌入向量與知識庫內容已成功儲存！")


處理 config.json 時發生錯誤：string indices must be integers, not 'str'


Loop thread traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 619, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/loc

OpenAI API 錯誤：HTTP code 520 from API (<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.openai.com | 520: Web server is returning an unknown error</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2

Loop thread traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 619, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/loc

KeyboardInterrupt: 

In [5]:
import discord
from discord.ext import commands
import openai
import json
import faiss
import numpy as np
import os
import nest_asyncio
import asyncio

# 定義 JSON 配置檔案名稱與其他檔案路徑
CONFIG_FILE = '/content/drive/MyDrive/Colab Notebooks/DC_BOT/config.json'
EMBEDDINGS_FILE = 'embeddings.npy'  # 嵌入向量檔案名稱

# 讀取 JSON 檔案的共用函式
def load_json(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"錯誤: 找不到 {file_path}。請確保檔案存在。")
        exit(1)
    except json.JSONDecodeError:
        print(f"錯誤: 無法解析 {file_path} 的 JSON 格式。請檢查檔案格式。")
        exit(1)

# 載入配置
config = load_json(CONFIG_FILE)
DISCORD_BOT_TOKEN = config.get('discord_bot_token')
OPENAI_API_KEY = config.get('openai_api_key')
KNOWLEDGE_BASE_FOLDER = config.get('knowledge_base_folder', '/content/drive/MyDrive/Colab Notebooks/DC_BOT/knowledge_base/')

if not DISCORD_BOT_TOKEN or not OPENAI_API_KEY:
    print("錯誤: config.json 中缺少必要的配置值。")
    exit(1)

openai.api_key = OPENAI_API_KEY

# 載入知識庫
def load_knowledge_base(folder_path):
    knowledge_base = []
    if not os.path.isdir(folder_path):
        print(f"錯誤: {folder_path} 不是一個有效的資料夾路徑。")
        exit(1)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            try:
                data = load_json(file_path)
                for item in data:
                    knowledge_base.append(f"{item['act']} {item['article']}: {item['content']}")
            except KeyError as e:
                print(f"{file_name} 缺少鍵: {e}")
            except Exception as e:
                print(f"處理 {file_name} 時發生錯誤: {e}")
    return knowledge_base

knowledge_base = load_knowledge_base(KNOWLEDGE_BASE_FOLDER)
if not knowledge_base:
    print("錯誤: 未能從知識庫中讀取到任何內容。")
    exit(1)

if not os.path.exists(EMBEDDINGS_FILE):
    print(f"錯誤: 找不到嵌入向量檔案 {EMBEDDINGS_FILE}。")
    exit(1)
embeddings = np.load(EMBEDDINGS_FILE).astype('float32')

# 建立 FAISS 索引
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print("FAISS 索引建立成功。")

# 初始化 Discord 客戶端
intents = discord.Intents.default()
intents.message_content = True
bot = commands.Bot(command_prefix="!", intents=intents)

@bot.event
async def on_ready():
    print(f'已登入為 {bot.user}')

@bot.command()
async def call(ctx, *, user_message: str):
    try:
        response = openai.Embedding.create(
            input=[user_message],
            model="text-embedding-ada-002"
        )
        query_embedding = response['data'][0]['embedding']
    except openai.OpenAIError as e:
        await ctx.send("抱歉，無法生成您的請求的嵌入向量。")
        print(f"OpenAI API 錯誤: {e}")
        return

    # 使用 FAISS 搜尋最相關的 3 筆資料
    D, I = index.search(np.array([query_embedding]).astype('float32'), k=3)
    print("FAISS 搜尋結果索引:", I[0])
    for i, idx in enumerate(I[0]):
        print(f"搜尋結果 {i+1}: {knowledge_base[idx]}")

    retrieved_docs = [knowledge_base[i].strip() for i in I[0]]
    context_text = "\n".join(retrieved_docs)

    prompt = f"根據以下內容回答問題：\n{context_text}\n\n問題：{user_message}\n回答："

    try:
        completion = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "你是一個律師，專門提供法律諮詢。"},
                {"role": "user", "content": prompt}
            ],
            max_tokens=600,
            temperature=0.7
        )
        reply = completion.choices[0].message.content
    except openai.OpenAIError as e:
        reply = "抱歉，我無法處理您的請求。"
        print(f"OpenAI API 錯誤: {e}")

    await ctx.send(reply)

async def monitor_exit():
    while True:
        cmd = await asyncio.to_thread(input, "請輸入指令 (輸入 'exit' 可關閉程式): ")
        if cmd.strip().lower() == "exit":
            print("接收到 exit 指令，正在關閉 Bot...")
            await bot.close()
            break

nest_asyncio.apply()

if __name__ == "__main__":
    async def main():
        monitor_task = asyncio.create_task(monitor_exit())
        bot_task = asyncio.create_task(bot.start(DISCORD_BOT_TOKEN))
        done, pending = await asyncio.wait(
            [monitor_task, bot_task],
            return_when=asyncio.FIRST_COMPLETED
        )
        for task in pending:
            task.cancel()

    asyncio.run(main())


處理 config.json 時發生錯誤: string indices must be integers, not 'str'
FAISS 索引建立成功。
已登入為 JiaHong#8858
FAISS 搜尋結果索引: [104 101 103]
搜尋結果 1: 道路交通管理處罰條例 第 55 條: 1. 汽車駕駛人，臨時停車有下列情形之一者，處新臺幣三百元以上六百元以下罰鍰：
一、在橋樑、隧道、圓環、障礙物對面、人行道、行人穿越道、快車道臨時停車。
二、在交岔路口、公共汽車招呼站十公尺內或消防車出、入口五公尺內臨時停車。
三、在設有禁止臨時停車標誌、標線處所臨時停車。
四、不依順行之方向，或不緊靠道路右側，或單行道不緊靠路邊，或併排臨時停車。
五、在道路交通標誌前臨時停車，遮蔽標誌。
2. 接送未滿七歲之兒童、行動不便之人上、下車者，臨時停車不受三分鐘之限制。
搜尋結果 2: 道路交通管理處罰條例 第 53 條: 1. 汽車駕駛人，行經有燈光號誌管制之交岔路口闖紅燈者，處新臺幣一千八百元以上五千四百元以下罰鍰。
2. 前項紅燈右轉行為者，處新臺幣六百元以上一千八百元以下罰鍰。
搜尋結果 3: 道路交通管理處罰條例 第 54 條: 汽車駕駛人，駕車在鐵路平交道有下列情形之一者，處新臺幣一萬五千元以上九萬元以下罰鍰，並吊扣其駕駛執照一年。因而肇事者，吊銷其駕駛執照：
一、不遵守看守人員之指示，或警鈴已響、閃光號誌已顯示，或遮斷器開始放下，仍強行闖越。
二、在無看守人員管理或無遮斷器、警鈴及閃光號誌設備之鐵路平交道，設有警告標誌或跳動路面，不依規定暫停，逕行通過。
三、在鐵路平交道超車、迴車、倒車、臨時停車或停車。


KeyboardInterrupt: 